# NOTE PATHS HAVE NOT BEEN CHANGED SINCE MOVING INTO NEW FOLDER

In [13]:
import numpy as np
import pandas as pd
import re

In [14]:
folder = "data"
file = folder + '/elevation_10km.csv'

df = pd.read_csv(file)

In [15]:
long = []
lat = []
for i, row in df.iterrows():
    coords = re.search(r'\[(.*?)\]', row['.geo']).group(1).split(',')
    long.append(float(coords[0]))
    lat.append(float(coords[1]))

df['long'] = long
df['lat'] = lat
df.drop(columns=['.geo', 'system:index'], axis=1, inplace=True)

In [16]:
print("rows by columns:", df.shape)
print("columns:", df.columns)
print("longitude range:", max(df['long']), min(df['long']))
print("latitude range:", max(df['lat']), min(df['lat']))
print("elevation range:", max(df['elevation']), min(df['elevation']))

rows by columns: (12700, 3)
columns: Index(['elevation', 'long', 'lat'], dtype='object')
longitude range: 153.3873347634083 145.75165484839235
latitude range: -23.490944679725487 -38.492809924521495
elevation range: 1792.459 -0.39861158


In [17]:
lapse_rate = 0.0065 # deg C per m
sea_level_temp = 19 # deg C - along east coast, using Sydney as reference
err_std_dev = 0.2 # deg C - taking into account errors for lapse rate and sea level temp

def elevation_to_temp(elevations): # elevation in meters
    temps = []
    for elevation in elevations:
        err = np.random.normal(0, err_std_dev)
        temp = sea_level_temp - (lapse_rate * elevation) + err
        temps.append(temp)
    return temps

In [18]:
df['generated_temp'] = elevation_to_temp(df['elevation'])
df.to_csv(folder + '/10km_data.csv', index=False)